In [1]:
!nvidia-smi

Mon May  6 15:47:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              48W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!git clone https://github.com/SkalskiP/yolov9.git


Cloning into 'yolov9'...
remote: Enumerating objects: 325, done.
remote: Counting objects: 100% (216/216), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 325 (delta 158), reused 155 (delta 155), pack-reused 109
Receiving objects: 100% (325/325), 2.23 MiB | 15.57 MiB/s, done.
Resolving deltas: 100% (165/165), done.


In [4]:
%cd yolov9


/content/yolov9


In [5]:
!pip install -r requirements.txt -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.1 MB/s eta 0:00:00


### Downloading Weights:

In [6]:
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt

#i have trained these one
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-c.pt


!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-e.pt

# Dataset:

In [7]:
!unzip '/content/drive/MyDrive/strokes_detection-20240503T132808Z-001/strokes_detection/Face-Stroke-Detector-1-20240503T055635Z-001.zip'

Archive:  /content/drive/MyDrive/strokes_detection-20240503T132808Z-001/strokes_detection/Face-Stroke-Detector-1-20240503T055635Z-001.zip
  inflating: Face-Stroke-Detector-1/data.yaml  
  inflating: Face-Stroke-Detector-1/README.roboflow.txt  
  inflating: Face-Stroke-Detector-1/README.dataset.txt  
  inflating: Face-Stroke-Detector-1/test/images/img_2850_jpg.rf.747d58f7b259fef5f68a237c29cec705.jpg  
  inflating: Face-Stroke-Detector-1/test/images/img_3216_jpg.rf.ba55021b32967f0ce2332f2f5e496f75.jpg  
  inflating: Face-Stroke-Detector-1/test/images/img_3103_jpg.rf.350a13994f7a86f26a888adf104ec810.jpg  
  inflating: Face-Stroke-Detector-1/test/images/img_3154_jpg.rf.9e54411b193bfb42d1c3127a523015bb.jpg  
  inflating: Face-Stroke-Detector-1/test/images/img_3206_jpg.rf.692c85299c0bb77631b421b96cd08474.jpg  
  inflating: Face-Stroke-Detector-1/test/images/img_2796_jpg.rf.33d441e696b56ccbb62172c44d7d1aec.jpg  
  inflating: Face-Stroke-Detector-1/test/images/img_3526_jpg.rf.2d0d95387483b5817

# Model Training:

In [8]:
!python train_dual.py \
--batch 32 --epochs 80 --img 240 --min-items 0 --close-mosaic 15 \
--data /content/yolov9/Face-Stroke-Detector-1/data.yaml \
--weights {HOME}/weights/yolov9-e.pt \
--cfg models/detect/yolov9-e.yaml \
--hyp hyp.scratch-high.yaml


2024-05-06 15:49:25.255201: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-06 15:49:25.255257: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-06 15:49:25.257152: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-06 15:49:26.310605: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
train_dual: weights={HOME}/weights/yolov9-e.pt, cfg=models/detect/yolov9-e.yaml, data=/content/yolov9/Face-Stroke-Detector-1/data.yaml, hyp=hyp.scratch-high.yaml, epochs=80, batch_size=32, imgsz=240, rect=False, resume=False, nosave=False, n

In [9]:
%pwd

'/content/yolov9'

In [10]:

def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()
imShow('/content/yolov9/runs/train/exp2/results.png')

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
imShow('/content/yolov9/runs/train/exp/P_curve.png')

# Inference:

In [ ]:
!python detect.py \
--img 256 --conf 0.1 --device 0 \
--weights /content/yolov9/runs/train/exp2/weights/best.pt \
--source /content/yolov9/Face-Stroke-Detector-1/test/images/img_0001_jpg.rf.b00a61e92f2e9b3464a779f0c3cb2a34.jpg

In [ ]:
!python detect.py \
--img-size 256 \
--conf 0.1 \
--device 0 \
--weights /content/yolov9/runs/train/exp/weights/best.pt \
--source /content/yolov9/Face-Stroke-Detector-1/test/images/img_0001_jpg.rf.b00a61e92f2e9b3464a779f0c3cb2a34.jpg


Prediction

In [ ]:
imShow('/content/yolov9/runs/detect/exp2/img_0001_jpg.rf.b00a61e92f2e9b3464a779f0c3cb2a34.jpg')

In [ ]:
!cp /content/yolov9/runs/train/exp2/weights/best.pt /content/drive/MyDrive/strokes_detection

# Plot:

In [ ]:
import pandas
import matplotlib.pyplot as plt

result = pandas.read_csv("/content/yolov9/runs/train/exp2/results.csv")

In [ ]:
result.columns

In [ ]:
train_loss = 7.5 * result['      train/box_loss'] + 0.5 * result['      train/cls_loss'] + 1.5 * result['      train/dfl_loss']
val_loss = 7.5 * result['        val/box_loss'] + 0.5 * result['        val/cls_loss'] + 1.5 * result['        val/box_loss']
xmax = len(val_loss)
plt.plot(train_loss.values, label="train")
plt.plot(val_loss.values, label="val")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.xlim([0, xmax])
plt.legend()
plt.grid()
plt.show()

In [ ]:

precision = result['   metrics/precision']
recall = result['      metrics/recall']
F1 = 2 * precision * recall / (precision + recall)
mAP = result['     metrics/mAP_0.5']

plt.plot(precision.values, label="Precision")
plt.plot(recall.values, label="Recall")
plt.plot(F1, label="F1")
plt.plot(mAP.values, label="mAP")
plt.xlabel("Epoch")
plt.xlim([0, xmax])
plt.legend()
plt.grid()
plt.show()


# Inference Time:

In [ ]:
import time
from PIL import Image

sizes = [(416, 416), (224, 224), (640, 480)]
inference_times = []

original_image = Image.open("/content/yolov9/runs/detect/exp/img_0001_jpg.rf.b00a61e92f2e9b3464a779f0c3cb2a34.jpg")

for size in sizes:
    start_time = time.time()

    resized_image = original_image.resize(size)

    end_time = time.time()

    inference_time_ms = (end_time - start_time) * 1000
    inference_times.append(inference_time_ms)
    print(f"Inference Time for size {size}: {inference_time_ms:.2f} ms")

plt.figure(figsize=(7, 5), facecolor='white')
image_sizes = [f"{size[0]}x{size[1]}" for size in sizes]
colors = inference_times

plt.scatter(image_sizes, inference_times, c=colors, cmap='viridis', s=100)
plt.colorbar(label='Inference Time (milliseconds)')

plt.title("Inference Time vs. Image Size")
plt.xlabel("Image Size (pixels)")
plt.ylabel("Inference Time (milliseconds)")
plt.grid(True)
plt.show()
